# 동화 텍스트 생성 AI

## 검증 -> 통합 프롬프트 재구성 -> LLM 생성 로직 구현

1단계: 라이브러리 설치 및 설정

In [10]:
# 1. 라이브러리 설치
!pip install openai -q

import os
from openai import OpenAI
import json
from getpass import getpass

# 2. API Key 입력 (실행 시 입력창에 sk-로 시작하는 키를 넣고 엔터를 치세요)
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key를 입력하세요: ")

client = OpenAI()

print("✅ 설정이 완료되었습니다.")

OpenAI API Key를 입력하세요: ··········
✅ 설정이 완료되었습니다.


2단계: 동화 생성 엔진 (아키텍처 반영 로직)

In [11]:
def generate_fairy_tale(user_input):
    print(f"\n🚀 [프로세스 시작] 사용자 입력: \"{user_input}\"")

    # ------------------------------------------------------------------
    # Step 1: 입력값 검증 및 분석 (Safety & Requirement Check)
    # ------------------------------------------------------------------
    print("1️⃣ [검증 단계] 입력값의 안전성과 적합성을 판단 중...")

    check_prompt = f"""
    당신은 동화 서비스의 '안전 관리자'입니다.
    사용자의 입력이 동화를 만들기에 적절한지 판단하세요.
    폭력적, 선정적, 혐오 표현이 있거나 동화와 전혀 관련 없는 요청이라면 'FALSE'를,
    적절하다면 'TRUE'를 출력하세요.
    설명 없이 오직 TRUE 또는 FALSE만 대답하세요.

    사용자 입력: {user_input}
    """

    check_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a safety moderator."},
                  {"role": "user", "content": check_prompt}],
        temperature=0.0
    )

    is_safe = check_response.choices[0].message.content.strip()

    if "FALSE" in is_safe:
        return "⚠️ [차단됨] 부적절하거나 동화 생성과 관련 없는 요청입니다. 다시 입력해주세요."

    print("   -> ✅ 검증 통과.")


    # ------------------------------------------------------------------
    # Step 2: 통합 프롬프트 재구성 (Prompt Re-engineering)
    # 사용자의 짧은 입력을 상세한 '동화 기획서'로 변환합니다.
    # ------------------------------------------------------------------
    print("2️⃣ [기획 단계] 사용자 의도를 분석하여 '통합 프롬프트' 생성 중...")

    architect_prompt = f"""
    당신은 전문 '동화 기획자'입니다.
    사용자의 입력을 분석하여 동화 작가가 글을 쓸 수 있도록 상세한 [동화 설계도]를 작성해야 합니다.
    사용자가 구체적으로 말하지 않은 부분은 문맥에 맞게 가장 매력적인 설정으로 채워넣으세요.

    [필수 분석 항목]
    1. 주인공 이름 및 특징 (없으면 어울리는 이름 생성)
    2. 핵심 주제 및 교훈
    3. 배경 (장소, 시간)
    4. 분위기 (예: 몽환적인, 신나는, 차분한 파스텔톤의)
    5. 포함된 문화 요소 (한국 전래동화 풍, 서양 판타지 풍 등)
    6. 줄거리 개요 (기승전결)

    [사용자 입력]
    {user_input}

    [출력 포맷 - JSON 형식만 출력할 것]
    {{
      "title": "동화 제목",
      "characters": "주인공과 등장인물 상세 설정",
      "setting": "배경 설정",
      "tone": "분위기",
      "moral": "교훈",
      "plot_outline": "줄거리 요약"
    }}
    """

    plan_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a creative story architect. Output JSON only."},
                  {"role": "user", "content": architect_prompt}],
        response_format={ "type": "json_object" },
        temperature=0.7
    )

    story_plan = json.loads(plan_response.choices[0].message.content)

    print("\n   📄 [생성된 동화 설계도]")
    print(f"   - 제목: {story_plan['title']}")
    print(f"   - 등장인물: {story_plan['characters']}")
    print(f"   - 교훈: {story_plan['moral']}")
    print(f"   - 분위기: {story_plan['tone']}")


    # ------------------------------------------------------------------
    # Step 3: 동화 텍스트 생성 (LLM Generation)
    # ------------------------------------------------------------------
    print("\n3️⃣ [생성 단계] 설계도를 바탕으로 GPT-4o가 동화를 집필 중입니다...")

    writer_prompt = f"""
    당신은 따뜻하고 교훈적인 이야기를 만드는 베스트셀러 동화 작가입니다.
    아래의 [동화 설계도]를 바탕으로 아이들이 읽기 좋은 동화를 작성해 주세요.

    [작성 가이드]
    1. 문체: '~했어요', '~였답니다'와 같은 부드러운 구어체(경어)를 사용하세요.
    2. 구성: 도입, 전개, 위기, 절정, 결말의 구조를 갖추세요.
    3. 분량: 읽어주었을 때 3분~5분 정도의 길이 (약 1500자 내외).
    4. 강조: 아이의 상상력을 자극하는 의성어, 의태어를 적절히 섞어주세요.

    [동화 설계도]
    {json.dumps(story_plan, ensure_ascii=False)}
    """

    story_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a warm and creative fairy tale writer."},
                  {"role": "user", "content": writer_prompt}],
        temperature=0.8
    )

    final_story = story_response.choices[0].message.content

    return final_story, story_plan

# 함수 테스트
print("✅ 동화 생성 엔진 준비 완료")

✅ 동화 생성 엔진 준비 완료


## 생성 → 후처리 검증(유해성/저작권/품질) → (미통과 시) 피드백 루프 → 최종 포맷팅 구현

3단계: 후처리 검증 및 재생성

In [12]:
import json

# ==============================================================================
# Step 4: 후처리 검증 (Editor AI)
# ==============================================================================
def verify_story_quality(story_text, story_plan):
    print("\n🧐 [검증 단계] '편집장 AI'가 원고를 심사 중입니다...")

    verification_prompt = f"""
    당신은 깐깐한 아동 문학 편집장입니다.
    동화 작가가 써온 원고를 아래 3가지 기준으로 심사하고, 합격 여부를 결정해주세요.

    [검증 기준]
    1. 유해성 (Safety): 아이들에게 공포감, 폭력성, 선정성, 혐오 표현을 주는 내용이 없는가?
    2. 저작권 (Copyright): 기존 유명 작품(해리포터 등)의 고유명사나 스토리를 베끼지 않았는가?
    3. 품질 (Quality):
       - 기획안의 교훈('{story_plan['moral']}')이 잘 드러나는가?
       - 한국적인 정서(예절, 가족애 등)나 아이들 눈높이에 맞는 표현이 사용되었는가?

    [원고 내용]
    {story_text}

    [출력 포맷 - JSON 형식만 출력]
    {{
        "is_passed": true 또는 false,
        "feedback": "통과했다면 null, 실패했다면 구체적인 수정 요청 사항",
        "check_results": {{
            "safety": "Pass/Fail",
            "copyright": "Pass/Fail",
            "quality": "Pass/Fail"
        }}
    }}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a strict editor. Output JSON only."},
                  {"role": "user", "content": verification_prompt}],
        response_format={ "type": "json_object" },
        temperature=0.0
    )

    return json.loads(response.choices[0].message.content)

# ==============================================================================
# Step 5: 피드백 반영 재작성 (Refinement)
# ==============================================================================
def refine_story(previous_story, feedback, story_plan):
    print(f"\n✍️ [수정 단계] 작가 AI가 피드백을 반영하여 원고를 다시 쓰고 있습니다...")

    refine_prompt = f"""
    당신은 동화 작가입니다. 초안이 편집 심사에서 반려되었습니다.
    아래 피드백을 반영하여 동화를 다시 작성해주세요.

    [기획 정보]
    제목: {story_plan['title']}
    교훈: {story_plan['moral']}

    [편집장 피드백 (수정 요청)]
    {feedback}

    [기존 원고]
    {previous_story}

    위 내용을 바탕으로 피드백을 해결한 완성된 동화를 다시 써주세요.
    (문체와 분량은 기존 가이드를 따르세요.)
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a professional fairy tale writer."},
                  {"role": "user", "content": refine_prompt}],
        temperature=0.7
    )

    return response.choices[0].message.content

# ==============================================================================
# Step 6: 최종 포맷팅 (Formatter)
# ==============================================================================
def format_final_output(valid_story, story_plan):
    # DB 스키마에 맞춘 최종 JSON 구조 생성
    final_data = {
        "title": story_plan['title'],
        "content": valid_story,
        "summary": story_plan['plot_outline'],
        "category": "창작동화",
        "moral": story_plan['moral'],
        "characters": story_plan['characters'],
        "tags": [story_plan['tone'], "교훈", "어린이"],
        "verification_status": "Verified"
    }
    return final_data

4단계: 메인 실행기-전체 프로세스 통합 및 출력 제어

In [13]:
# ==============================================================================
# [메인 실행기] 전체 프로세스 통합 및 출력 제어
# ==============================================================================
def run_process_with_display(user_input):
    # 1. 초안 생성 (Step 1~3)
    # (앞서 작성한 generate_fairy_tale 함수를 호출합니다)
    current_story, story_plan = generate_fairy_tale(user_input)

    print("\n" + "="*60)
    print("📜 [Step 1] 초안 생성 결과")
    print("="*60)
    print(current_story)

    max_retries = 3
    is_finalized = False

    # 2. 검증 및 수정 루프 (Step 4~5)
    for i in range(max_retries):
        print("\n" + "-"*60)
        print(f"🔄 [Step 2-{i+1}] 검증 및 수정 시도 ({i+1}/{max_retries})")
        print("-"*60)

        # 검증 수행
        verification = verify_story_quality(current_story, story_plan)

        if verification['is_passed']:
            print(f"   -> ✅ 심사 통과! (모든 기준 만족)")
            print(f"   -> 상세 결과: {verification['check_results']}")
            is_finalized = True
            break
        else:
            print(f"   -> ❌ 심사 반려")
            print(f"   -> 이유: {verification['feedback']}")
            print(f"   -> 상세 결과: {verification['check_results']}")

            # 수정 수행 (재생성)
            current_story = refine_story(current_story, verification['feedback'], story_plan)

            print("\n   [재생성된 텍스트 확인]")
            print("   " + "-"*30)
            print(current_story[:300] + "...\n   (중략)...") # 너무 길어지지 않게 앞부분만 표시 (필요시 전체 출력 가능)
            print("   " + "-"*30)

    # 3. 최종 결과 출력 (Step 6)
    if is_finalized:
        print("\n" + "="*60)
        print("📖 [Step 3] 최종 확정 텍스트")
        print("="*60)
        print(current_story)

        # JSON 변환
        final_json = format_final_output(current_story, story_plan)

        print("\n" + "="*60)
        print("💾 [Step 4] JSON 데이터 변환 결과")
        print("="*60)
        print(json.dumps(final_json, indent=4, ensure_ascii=False))

        return final_json
    else:
        print("\n⚠️ 수정 한도를 초과하여 최종 생성에 실패했습니다.")
        return None

실행 테스트

In [14]:
user_topic = input("동화 주제를 입력하세요: ")
if user_topic:
    final_output = run_process_with_display(user_topic)

동화 주제를 입력하세요: 동대문에서 공주님 옷을 찾는 유민이

🚀 [프로세스 시작] 사용자 입력: "동대문에서 공주님 옷을 찾는 유민이"
1️⃣ [검증 단계] 입력값의 안전성과 적합성을 판단 중...
   -> ✅ 검증 통과.
2️⃣ [기획 단계] 사용자 의도를 분석하여 '통합 프롬프트' 생성 중...

   📄 [생성된 동화 설계도]
   - 제목: 유민이와 동대문의 마법 옷
   - 등장인물: {'main_character': {'name': '유민이', 'description': '활발하고 상상력이 풍부한 10살 소녀로, 동대문 시장을 좋아하고 새로운 모험을 즐기는 성격이다.'}, 'supporting_characters': [{'name': '마법 재봉사 할머니', 'description': '동대문 시장에서 오래된 가게를 운영하며, 특별한 옷을 만들어주는 친절한 할머니.'}, {'name': "수다쟁이 고양이 '미야'", 'description': '유민이의 친구로, 언제나 유민이와 동대문 시장을 함께 탐험하며 유민이에게 조언을 해주는 존재이다.'}]}
   - 교훈: 소중한 것을 찾기 위해서는 자신의 마음을 따르고, 때로는 상상력과 모험심이 필요하다.
   - 분위기: 신비롭고 활기찬 분위기

3️⃣ [생성 단계] 설계도를 바탕으로 GPT-4o가 동화를 집필 중입니다...

📜 [Step 1] 초안 생성 결과
제목: 유민이와 동대문의 마법 옷

옛날 옛날, 아니 지금도, 서울의 동대문 시장엔 언제나 사람들로 북적북적했어요. 그곳에 사는 유민이는 상상력이 풍부한 10살 소녀였답니다. 여름 방학이 시작되자마자 유민이는 동대문 시장에서 자신만의 모험을 떠나기로 결심했어요. "이번엔 공주님 옷을 꼭 찾아내고 말 거야!" 유민이는 신이 나서 소리쳤답니다.

그날도 유민이는 수다쟁이 고양이 미야와 함께 시장으로 향했어요. 미야는 유민이의 든든한 친구였죠. 미야는 "유민아, 오늘은 어떤 모험이 우리를 기다리고 있을까?" 하고 물었어요. 두 친구는 시장을 이리저리